In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([2])


In [6]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [7]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [8]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [9]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.6270,  0.0447, -0.1165, -0.2724, -0.1388,  0.0334,  0.2701, -0.2049,
         -0.4169,  0.1632,  0.3514, -0.0519, -0.1324,  0.0488, -0.2011, -0.0685,
          0.4285, -0.1162, -0.0775, -0.5152],
        [-0.4929,  0.2704, -0.5485, -0.3765, -0.3905, -0.0311,  0.5354, -0.1695,
         -0.4270, -0.2386, -0.1911, -0.5002,  0.0909, -0.0095, -0.5704, -0.3083,
          0.0946, -0.5062, -0.1747, -0.1457],
        [-0.5107,  0.2466, -0.3627, -0.2226, -0.1619,  0.0921,  0.2136, -0.0211,
         -0.1413, -0.1551, -0.1261,  0.1179,  0.1033,  0.0715, -0.2278, -0.2281,
          0.1556, -0.4926, -0.3592, -0.2185]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0447, 0.0000, 0.0000, 0.0000, 0.0334, 0.2701, 0.0000, 0.0000,
         0.1632, 0.3514, 0.0000, 0.0000, 0.0488, 0.0000, 0.0000, 0.4285, 0.0000,
         0.0000, 0.0000],
        [0.0000, 0.2704, 0.0000, 0.0000, 0.0000, 0.0000, 0.5354, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0909, 0.0000, 0.00

In [10]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [11]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [12]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0327,  0.0174, -0.0223,  ...,  0.0147,  0.0277, -0.0108],
        [-0.0010,  0.0215, -0.0185,  ...,  0.0191, -0.0337,  0.0087]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0117, -0.0068], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0219,  0.0038, -0.0436,  ...,  0.0316,  0.0366,  0.0014],
        [ 0.0370, -0.0282, -0.0320,  ..., -0.0004, -0.0205, -0.0383]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | 